In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

# ch14. 웹데이터 수집
## 1절. BeautifulSoup과 parser
pip install bs4 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함

* 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [3]:
# 내 local pc의 파일을 url처럼 접근
import requests
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [4]:
s=requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://",FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬 파일)을 접근
response=s.get('file:///ai/lecNote/01_python/data/ch14_sample.html')
response


<Response [200]>

In [5]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다')

잘 접근하였습니다.


In [ ]:
response.status_code
    # 200: 정상
    # 404: 없는 페이지
    # 406: get,post 오류

In [6]:
response.content # 바이너리 형식의 html 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [7]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [8]:
response.text # html 파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [10]:
# html 파싱
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content,# response.text
                  "html.parser")
# soup


In [14]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

el : <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성 : text
el의 id속성 : text
el의 href속성 : None
el의 태그이름 : h1


In [13]:
# soup.select('선택자') : 해당 선택자 모두 
els = soup.select('h1') # h1태그를 list
print('els :', els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]


In [16]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [17]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [18]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [19]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class', 'css'}))
print('find(None) :', soup.find('a', {'class':'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터 수집
- 1) 환율 정보 가져오기(네이버증권-> 시장지표)
- https://finance.naver.com/marketindex/
    

In [10]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
 # Allow:/ - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [11]:
# 방법1
import requests
from bs4 import BeautifulSoup
url='https://finance.naver.com/marketindex/'
response=requests.get(url)
# response.status_code
# response.text/response.content.decode('cp949')
soup=BeautifulSoup(response.text,"html.parser")

In [12]:
# 방법2
from urllib.request import urlopen
url='https://finance.naver.com/marketindex/'
response=urlopen(url)
response.status
# response.read().decode / response.read().decode('cp949')
soup=BeautifulSoup(response,"html.parser")

In [13]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
    # print(price.text.replace(',', ''))
    prices.append(float(''.join(price.text.split(',') ) ) )
print(prices)

[1454.0, 942.93, 1682.13, 204.13, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1699.82, 4009.8, 190498.44]


In [14]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1454.0, 942.93, 1682.13, 204.13, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1699.82, 4009.8, 190498.44]


In [15]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
        print(t.text, end='\t')


미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [16]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 '' 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [17]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

하락	하락	하락	하락	상승	하락	상승	하락	상승	상승	상승	상승	

In [19]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [20]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,454.00원 - 하락
일본 JPY(100엔) : 942.93원 - 하락
유럽연합 EUR : 1,682.13원 - 하락
중국 CNY : 204.13원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1699.82원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 190498.44원 - 상승


In [21]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,454.00원 -하락
일본 JPY(100엔) : 942.93원 -하락
유럽연합 EUR : 1,682.13원 -하락
중국 CNY : 204.13원 -하락
달러/일본 엔 : 153.4200엔 -상승
유로/달러 : 1.1564달러 -하락
영국 파운드/달러 : 1.3161달러 -상승
달러인덱스 : 99.4700 -하락
WTI : 59.75달러 -상승
휘발유 : 1699.82원 -상승
국제 금 : 4009.8달러 -상승
국내 금 : 190498.44원 -상승


2)이번주 로또 번호 출력
https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
    1197회(2025년 11월 08일 추첨)
    당첨번호 [1, 5, 7, 26, 28, 43]
    보너스 30

## 2) 이번주 로또 번호 출력
* https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)

    1197회(2025년 11월 08일 추첨)
    
    당첨번호 [1, 5, 7, 26, 28, 43]
    
    보너스 30

In [22]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [23]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [24]:
times = soup.select_one('div.win_result strong').text # 1197회
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [25]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


In [43]:
pip install requests beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.


In [44]:
import requests
from bs4 import BeautifulSoup  # 1. import 추가

def get_latest_lotto_api():
    """동행복권 API를 사용한 최신 로또 번호 조회 (오류 처리 포함)"""
    try:
        # 2. URL 공백 제거
        main_url = 'https://dhlottery.co.kr/common.do?method=main'
        response = requests.get(main_url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")
        # 회차 요소가 없을 경우 대비
        round_element = soup.find('strong', id='lottoDrwNo')
        if not round_element:
            raise ValueError("회차 정보를 찾을 수 없습니다")
        
        latest_round = int(round_element.text)
        print(f"최신 회차: {latest_round}회")
        
        # 3. 도메인 통일 및 공백 제거
        api_url = f'https://dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={latest_round}'
        api_response = requests.get(api_url, timeout=10)
        api_response.raise_for_status()
        
        data = api_response.json()
        
        # 4. 응답 데이터 검증
        if data.get('returnValue') == 'fail':
            raise ValueError(f"해당 회차({latest_round}) 데이터가 없습니다")
        
        return {
            'round': data['drwNo'],
            'date': data['drwNoDate'],
            'numbers': [data[f'drwtNo{i}'] for i in range(1, 7)],
            'bonus': data['bnusNo']
        }
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 네트워크 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ 처리 오류: {e}")
        return None

# 실행 예시
if __name__ == "__main__":
    result = get_latest_lotto_api()
    if result:
        print(f"\n✅ 제 {result['round']}회 로또 당첨번호")
        print(f"추첨일: {result['date']}")
        print(f"번호: {result['numbers']}")
        print(f"보너스: {result['bonus']}")

최신 회차: 1197회

✅ 제 1197회 로또 당첨번호
추첨일: 2025-11-08
번호: [1, 5, 7, 26, 28, 43]
보너스: 30


In [41]:
# 방법 1: requests (추천)
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def get_latest_lotto_numbers():
    """
    이번주 최신 로또 당첨번호를 가져오는 함수
    """
    try:
        # 1. 최신 회차 번호 가져오기
        main_url = 'https://dhlottery.co.kr/common.do?method=main'
        response = requests.get(main_url)
        response.raise_for_status()  # 오류 체크
        
        soup = BeautifulSoup(response.text, "html.parser")
        latest_round = int(soup.find('strong', id='lottoDrwNo').text)
        
        # 2. 당첨 정보 페이지 접근
        url = f'https://dhlottery.co.kr/gameResult.do?method=byWin&drwNo={latest_round}'
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 3. 추첨일 추출
        draw_date = soup.find('p', class_='desc').text
        
        # 4. 당첨번호 추출
        win_numbers = []
        win_area = soup.find('div', class_='num win').find_all('span')
        for num in win_area:
            win_numbers.append(int(num.text))
        
        # 5. 보너스번호 추출
        bonus_number = int(soup.find('div', class_='num bonus').find('span').text)
        
        return {
            'round': latest_round,
            'date': draw_date,
            'numbers': win_numbers,
            'bonus': bonus_number
        }
        
    except requests.exceptions.RequestException as e:
        print(f"요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 처리 중 오류 발생: {e}")
        return None

# 실행 및 결과 출력
if __name__ == "__main__":
    print("=" * 40)
    print("이번주 로또 6/45 당첨번호")
    print("=" * 40)
    
    result = get_latest_lotto_numbers()
    if result:
        print(f"회차: 제 {result['round']}회")
        print(f"추첨일: {result['date']}")
        print(f"당첨번호: {result['numbers']}")
        print(f"보너스번호: {result['bonus']}")
        
        # 시각적 표현
        print("\n[번호 분포]")
        for i, num in enumerate(result['numbers'], 1):
            print(f"  {i}번째 숫자: {num:02d}")
        print(f"  보너스: {result['bonus']:02d}")

이번주 로또 6/45 당첨번호
회차: 제 1197회
추첨일: (2025년 11월 08일 추첨)
당첨번호: [1, 5, 7, 26, 28, 43]
보너스번호: 30

[번호 분포]
  1번째 숫자: 01
  2번째 숫자: 05
  3번째 숫자: 07
  4번째 숫자: 26
  5번째 숫자: 28
  6번째 숫자: 43
  보너스: 30



## 3) 다음 검색 리스트
no title               link
0  한풀 꺾인 비트코인  https://v.daum.net/v/20251110094711892

In [26]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# 방법2 : 불가
# from urllib.request import urlopen
# from urllib.parse import quote
# word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
# print(url)
# response = urlopen(url)
# soup = BeautifulSoup(response, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""돈 풀린다""...비트코인, 美 셧다운 해제 기대감에 5% 급등",http://v.daum.net/v/20251110164345783
1,1,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
2,2,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
3,3,[투자 노하우] 美 셧다운 종료 기대감…비트코인 10만6천달러선,http://v.daum.net/v/20251110165508316
4,4,"[자막뉴스] 올해 초까지 가장 핫했는데…""안전띠 꽉 매세요"" 투자자 '비명'",http://v.daum.net/v/20251110165704414
5,5,"""2000달러 관세 배당"" 비트코인에 불 지핀 트럼프…산타랠리 시작?",http://v.daum.net/v/20251110160448927
6,6,“지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인,http://v.daum.net/v/20251110114112103
7,7,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
8,8,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [27]:
items_find_list = [] # 검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크'])

,번호,제목,링크
0,0,"""돈 풀린다""...비트코인, 美 셧다운 해제 기대감에 5% 급등",http://v.daum.net/v/20251110164345783
1,1,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
2,2,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
3,3,[투자 노하우] 美 셧다운 종료 기대감…비트코인 10만6천달러선,http://v.daum.net/v/20251110165508316
4,4,"[자막뉴스] 올해 초까지 가장 핫했는데…""안전띠 꽉 매세요"" 투자자 '비명'",http://v.daum.net/v/20251110165704414
5,5,"""2000달러 관세 배당"" 비트코인에 불 지핀 트럼프…산타랠리 시작?",http://v.daum.net/v/20251110160448927
6,6,“지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인,http://v.daum.net/v/20251110114112103
7,7,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
8,8,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [28]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""돈 풀린다""...비트코인, 美 셧다운 해제 기대감에 5% 급등",http://v.daum.net/v/20251110164345783
1,1,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
2,2,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
3,3,[투자 노하우] 美 셧다운 종료 기대감…비트코인 10만6천달러선,http://v.daum.net/v/20251110165508316
4,4,"[자막뉴스] 올해 초까지 가장 핫했는데…""안전띠 꽉 매세요"" 투자자 '비명'",http://v.daum.net/v/20251110165704414
5,5,"""2000달러 관세 배당"" 비트코인에 불 지핀 트럼프…산타랠리 시작?",http://v.daum.net/v/20251110160448927
6,6,“지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인,http://v.daum.net/v/20251110114112103
7,7,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
8,8,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [60]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [1]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    response = requests.get(url)
    print(response.status_code)

In [62]:
collect_news_list('청바지', 2)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): search.daum.net:443
DEBUG:urllib3.connectionpool:https://search.daum.net:443 "GET /search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EC%B2%AD%EB%B0%94%EC%A7%80&p=2 HTTP/1.1" 200 None


200


In [2]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    print(keyword)
    for idx, item in enumerate(item_titles):
        print([(page-1)*10+idx, item.text, item.attrs['href']])
        

In [65]:
collect_news_list('비트코인', 3)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): search.daum.net:443
DEBUG:urllib3.connectionpool:https://search.daum.net:443 "GET /search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8&p=3 HTTP/1.1" 200 None


비트코인
[20, ' 비트코인, 위태로운 10만 달러..."하락장 전환"vs"숨고르기" ', 'http://v.daum.net/v/20251106140714590']
[21, ' 연구소서 실험하려 샀는데…비트코인 1000배 대박 터졌다 ', 'http://v.daum.net/v/20251109120511705']
[22, ' "비트코인 \'1000배 잭팟\' 터뜨렸어요"···실험용으로 97개 샀다가 \'횡재\' 맞은 \'이 나라\' ', 'http://v.daum.net/v/20251109075014647']
[23, ' 돈나무 언니도 생각 바꿨다…비트코인 강세전망 20%↓ ', 'http://v.daum.net/v/20251107040000760']
[24, ' JP모건 "비트코인, 금 비해 저평가...공정가치 17만 달러" ', 'http://v.daum.net/v/20251107092549359']
[25, ' 돈나무 언니, 비트코인 전망 120만달러로 낮췄다...왜? ', 'http://v.daum.net/v/20251107170008576']
[26, " 연구용으로 산 비트코인 97개…13년만에 1000배 수익 '잭팟' ", 'http://v.daum.net/v/20251109113613079']
[27, ' “주식보다 더 떨어져”…휘청이는 비트코인 “분위기 반전 어렵다” ', 'http://v.daum.net/v/20251106171049044']
[28, ' 한때 10만弗 붕괴, 날개 꺾인 비트코인 ', 'http://v.daum.net/v/20251106041647056']
[29, ' “곡소리 들리네” 비트코인·이더리움 붕괴…위험자산 ‘회피 심리’ ', 'http://v.daum.net/v/20251105080314455']


In [31]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [32]:
collect_news_list('AI', 1)

[[0,
  " 초중고 정보 교과 내 AI 교육 확대…모든 교육청 'AI 지원센터' 설립 ",
  'http://v.daum.net/v/20251110140139274'],
 [1,
  ' 또 퍼지는 ‘AI거품론’… 커지는 ‘AI워싱’ 경보 ',
  'http://v.daum.net/v/20251109183027380'],
 [2, ' AI 거품론 속…오픈AI, 백악관에 SOS ', 'http://v.daum.net/v/20251110042102550'],
 [3,
  " 무협, 'AI 서밋 서울 2025' 개막…글로벌 AI 기업 총집결 ",
  'http://v.daum.net/v/20251110164225706'],
 [4,
  " [뉴스1 PICK]국내 최대 AI 컨펙스 'AI 서밋 서울 & 엑스포' 개막 ",
  'http://v.daum.net/v/20251110145713498'],
 [5,
  ' 5.5년만에 AI 박사학위 취득...내년 AI인재양성에 1.4조 예산 투입 ',
  'http://v.daum.net/v/20251110142138252'],
 [6,
  ' 서울AI재단-MIT 센서블시티랩, 도시 AI 연구 협력 본격화 ',
  'http://v.daum.net/v/20251110173813202'],
 [7,
  ' "도메인 지식 없는 AI는 공허"···최태원, 본업 기반 AI 경쟁력 강조 ',
  'http://v.daum.net/v/20251110144852927'],
 [8,
  ' 야놀자, 첫 AI 해커톤 개최…"AI 혁신 전사적 전파할 것" ',
  'http://v.daum.net/v/20251110160146766'],
 [9,
  " 중견회계법인협의회, AI 감사 솔루션 '오딘 AI' 도입 ",
  'http://v.daum.net/v/20251110150515967']]

In [34]:
import time
keywords = ['청바지', '동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색 결과 수집 중입니다===
===2번째 검색어 동대문 검색 결과 수집 중입니다===


In [35]:
result0_df = pd.DataFrame(result0, columns=['no', 'title','link'])
result1_df = pd.DataFrame(result1, columns=['no', 'title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"동대문구, ‘11월 자전거교실’ 개강… 올해 마지막 초보자 교육 진행",http://v.daum.net/v/20251110145114101
2,2,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
3,3,"동대문구, 25일까지 '주민 건강도시학교'···""건강하게 살자""",http://v.daum.net/v/20251109223318356
4,4,"서울 동대문구, 15일 배봉산정원 가족축제 개최",http://v.daum.net/v/20251104104517122


In [36]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False, encoding='cp949')
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
* 자동으로 브라우저를 통해 요청하는 것처럼 보이기

In [89]:
# 방법 2:
from urllib.request import urlopen, Request
from urllib.parse import quote
word=quote('비트코인')
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
headers={ 'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'}
request=Request(url,headers=headers)
response=urlopen(request)
soup=BeautifulSoup(response, 'html.parser')
#soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


* https://www.melon.com/chart/index.htm

* https://wwww.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [37]:
# 방법2 : 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
# headers = {'User-Agent':
#           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)
request = Request(url)
request.add_header('User-Agent', 
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')

response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


* https://www.melon.com/chart/index.htm
* https://wwww.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [38]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup

<Response [406]>

In [39]:
# 방법2 : User-Agent 추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
# soup

In [40]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') # title.text.strip()
singers = soup.select('div.ellipsis.rank02') # singer.text.strip()[:20]
# 1위 | 노래제목 - 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Drowning - WOODZWOODZ
5위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
6위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위 | 달리 표현할 수 없어요 - 로이킴로이킴
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | Soda Pop - KPop Demon Hunters C
10위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
12위 | FAMOUS - ALLDAY PROJECTALLDAY
13위 | IRIS OUT - Kenshi YonezuKenshi 
14위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
15위 | XOXZ - IVE (아이브)IVE (아이브)
16위 | Rich Man - aespaaespa
17위 | 한번 더 이별 - 이창섭이창섭
18위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
19위 | 너에게 닿기를 - 10CM10CM
20위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | Whiplash - aespaaespa
25위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
26위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위 | 천상연 - 이창섭이창섭
28위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AKMU (악뮤)AKMU (악뮤)
29위 |

### 5) 네이버 지식인으로 검색(open API 사용x)
- 특정 keyword를 특정 페이지 수만큼

In [1]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
url = f'https://kin.naver.com/search/list.naver'
params = {'query' : keyword}
response = get(url, params=params)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
# soup

200


In [2]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
#soup

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [10]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = []
for page in range(1, pages+1):
    url=f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    respons=urlopen(url)
    # print(respons.status)
    soup=BeautifulSoup(respons,'html.parser')

200
200
200


In [12]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list(title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link  = item.attrs.get('href')
        items_list.append({
            'title': title,
            'link' : link
        })
df = pd.DataFrame(items_list)
print(df.shape)
df.sample()
(30, 2)

(30, 2)


(30, 2)

In [ ]:
title	link
29	쳇지피티 일러스트	https://kin.naver.com/qna/detail.naver?d1id=3&..

## 2.2 openAPI사용: json 웹데이터 수집
### 1) 네이버 지식인으로 검색(openAPI 사용)
- 네이버 개발자센터에서 애플리케이션등록(이름/검색/web설정 http://localhost
- .env 파일에 CLIENT_ID=KY_SuI6XpQ0cL7wz146v, CLIENT_SECRET=olGRlm4BqP, 환경변수 저장
- 환경변수를 읽기 위해서 `pip install dotenv'
_ 특정 keyword를 지식 검색(데이터수 30개)

In [18]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [22]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
    #dotenv_path='.env'
        ) # 현 소스와
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))


True

In [28]:
# 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)
rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])
else:
    print("Error Code:" + rescode)


<class 'str'>
{
	"lastBuildDate":"Tue, 11 Nov 2025 12:03:10 +090


In [25]:
# 방법 2
import os
import sys
import urllib.request
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers={
    X-Naver-Client-Id",client_id,
    X-Naver-Client-Secret",client_secret
}
request=urllib.request.Reqe
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Tue, 11 Nov 2025 11:23:54 +0900",
	"total":3143,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"<b>쳇지피티<\/b> 사주 정확성",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=3&dirId=31501&docId=489876742&qb=7LOH7KeA7ZS87Yuw&enc=utf8",
			"description":"<b>쳇지피티<\/b> 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요 안녕하세요. 요즘 지피티에 대해 물어보시는 분들이 많아서 저도 해봤지만 아직 생년월일도ㅠ파악하지 못합니다. 유료버전 무료버전 둘 다... "
		},
		{
			"title":"<b>쳇지피티<\/b> 관해서",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=1&dirId=10704&docId=488200030&qb=7LOH7KeA7ZS87Yuw&enc=utf8",
			"description":"<b>쳇지피티<\/b>한테 질문 제대로 하려면 어떻게 해야 하냐요?... <b>쳇지피티<\/b> 한테 하는질문도 1.우리 이야기 만들래)?(호칭.... <b>쳇지피티<\/b> 한테 어떻게 질문해야 정확한 답변을 들을수 있죠 <b>쳇지피티<\/b> 앱이 편하시다 하신분들은 편하신이유... "
		},
		{
			"title":"사주에서 뭐가 부족한지 봐주세요(<b>쳇지피티<\/b> 답변 금지)",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=3&dirId=31501&docId=488905210&qb=7LOH7KeA7ZS87Yuw&enc=utf8",
			"description":"사주에서 뭐가 부족한지 봐주세요(<b>쳇지피티<\/b> 답변 금지) 남매입니다

In [33]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=30'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 쳇지피티 어떻게 하는건가욤 유료결제인가요 유료결제는 얼마인가요 그거 하게되면...


### quiz) 네이버 open API를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지_csv 출력
```
제목, 링크, 썸네일, sizeheight, sizewidth

In [36]:
import requests
import pandas as pd
import os

# 1. 인증 정보 설정 (환경변수에서 가져오기)
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# 2. 검색어 및 API 파라미터 설정
query = '청바지'
url = 'https://openapi.naver.com/v1/search/image'
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}
params = {
    'query': query,
    'display': 100,
    'sort': 'sim'
}

# 3. API 호출
response = requests.get(url, params=params, headers=headers)

# 4. 응답 확인 및 데이터 처리
if response.status_code == 200:
    items = response.json()['items']
    
    items_list = []
    for item in items:
        title = item.get('title', '').replace('<b>', '').replace('</b>', '').strip()
        link = item.get('link', '')
        thumbnail = item.get('thumbnail', '')
        sizeheight = item.get('sizeheight', '')
        sizewidth = item.get('sizewidth', '')
        
        items_list.append([title, link, thumbnail, sizeheight, sizewidth])
    
    # 5. DataFrame 생성 및 CSV 저장
    df = pd.DataFrame(items_list, columns=['제목', '링크', '썸네일', 'sizeheight', 'sizewidth'])
    df.to_csv('ch14_청바지.csv', index=False, encoding='utf-8-sig')
    
    print(f"✅ 성공! {len(df)}건의 청바지 이미지 데이터를 'ch14_청바지.csv'로 저장했습니다.")
    print(df.head(3))
    
else:
    print(f"❌ 오류 발생! 상태 코드: {response.status_code}")
    print(response.text)

✅ 성공! 100건의 청바지 이미지 데이터를 'ch14_청바지.csv'로 저장했습니다.
                                                  제목  \
0  여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음...   
1                    -30 여자 청바지 여성 데님 밴딩 일자 통 와이드 팬츠   
2                        키작녀청바지 세미와이드청바지 진청바지 : 미니팬츠   

                                                  링크  \
0  https://shop-phinf.pstatic.net/20251015_281/17...   
1  http://shopping.phinf.naver.net/main_5666884/5...   
2  http://shop1.phinf.naver.net/20240320_292/1710...   

                                                 썸네일 sizeheight sizewidth  
0  https://search.pstatic.net/common/?type=b150&s...       1000      1000  
1  https://search.pstatic.net/common/?type=b150&s...        600       600  
2  https://search.pstatic.net/common/?type=b150&s...       1000      1000  


In [37]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
54,버버리 BURBERRY 패셔너블한 클래식 캐주얼 다용도 청바지 - 위핑 버버리 BU...,https://weping.co.kr/data/editor/goods/2530/20...,https://search.pstatic.net/sunny/?type=b150&sr...,1067,800


In [41]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    #print(items[0])
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
59,아가방 청바지 가을 로이일자핏데님바지 01R532001 50% : 아가방갤러리 에덴점,http://shop1.phinf.naver.net/20240727_152/1722...,https://search.pstatic.net/common/?type=b150&s...,4000,3000


In [39]:
print(df.loc[0, '링크'])
print(df.loc[0, '썸네일'])

https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png
https://search.pstatic.net/common/?type=b150&src=https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png


In [55]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import requests
    file_extension = link.split('.')[-1] # 확장자
    quote_index = file_extension.find('?') # 확장자뒤에 ?가 있는 위치 .jpg?w=780
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content # 바이너리
    with open(f'image/{attr}_{idx+1:02}_{query}.{file_extension}', 'wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[25, '링크'], '청바지')

In [56]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행중 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

= = = 20% 진행중 = = =
= = = 40% 진행중 = = =
= = = 60% 진행중 = = =
= = = 80% 진행중 = = =
= = = 완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
67,남자와이드청바지 남성더블턱청바지 : 설렌다,http://shop1.phinf.naver.net/20251010_63/17600...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


## 2.3 XML 웹데이터 수집
- RSS /open API을 통한 XML 웹데이터 수짐
### 1) 전국 날씨를 RSS를 BeautifalSoup을 이용한 크롤링
- 기상청 RSS 검색한 첫번째 사이트로 이동

In [60]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml") # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
pd.DataFrame(items_list)


,지역,1주평년기온,1주기온범위,1주낮을확률,1주비슷할확률,1주높을확률
0,"전국(제주도,북한제외)",5.9,5.0~6.8,20,40,40
1,서울ㆍ인천ㆍ경기도,4.8,3.7~5.9,20,40,40
2,강원도 영서,2.6,1.5~3.7,20,40,40
3,강원도 영동,5.8,4.9~6.7,20,40,40
4,대전ㆍ세종ㆍ충청남도,5.4,4.5~6.3,20,40,40
5,충청북도,4.1,3.1~5.1,20,40,40
6,광주ㆍ전라남도,8.3,7.4~9.2,20,40,40
7,전북자치도,5.9,4.9~6.9,20,40,40
8,부산ㆍ울산ㆍ경상남도,7.9,7.0~8.8,20,40,40
9,대구ㆍ경상북도,5.7,4.8~6.6,20,40,40


### 2) XML 응답하는 open API 활용
- data.go.kr에서 
    - 서울특별시_노선정보조회 서비스(버스 ID, 정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [ ]:
# STEP 1 버스번호의 버스 id 받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusLouteList) 이용

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))

True

In [3]:
# import requests
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
busNum='273'
key=os.getenv('KEY')
url=f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
savefilename1='data.ch14_busInfo.xml'
urlretrieve(url,savefilename1) # xml 파일(url)을 savefilename으로 저장
with open(savefilename1,encoding='utf-8') as f:
    xml=fread();
soup=BeautifulSoup(xml,'xml')

NameError: name 'fread' is not defined

In [4]:
import requests
from urllib.parse import quote
# busNum=quote('구로09')
busNum = '162'
key=os.getenv('KEY')
url=f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm==busNum:
        busRouteId=item.select_one('busRouteId').text
        break;
print('busRouteId=', busRouteId)

busRouteId= 100100034


In [5]:

# step2. 버스id로 버스정류장목록받아오기(정류장명, 정류장id, 경도, 위도)
# 서울특별시_노선정보조회 서비스 - 4번 기능(getStaionsByRouteList)

In [6]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소 이름
    station   = itemList.find('station').text   # 정류소 id
    gpsX      = itemList.find('gpsX').text      # 경도
    gpsY      = itemList.find('gpsY').text      # 위도
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명', 'id', '경도', '위도'])
df_station.head()

162번 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


In [7]:
# step3. 버스id로 실시간위치정보를 받아오기(차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)
# 서울특별시_버스위치정보조회 서비스 - 2번(getBusPosByRtidList)

In [10]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    congetion = itemList.select_one('congetion').text # 혼잡도(congetion)
    # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion=='0' \
            else '여유' if congetion=='3' \
            else '보통' if congetion=='4' \
            else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = itemList.select_one('gpsX').text # 경도
    gpsY = itemList.select_one('gpsY').text # 위도
    lastStnId = itemList.select_one('lastStnId').text # 최종정류소id
    nextStId = itemList.select_one('nextStId').text # 다음정류소id
    nextStTm = itemList.select_one('nextStTm').text # 다음정류소까지 소요시간
    
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion,
        '경도':gpsX,
        '위도':gpsY,
        '최종정류소id':lastStnId,
        '다음정류소id':nextStId,
        '도착소요시간':nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position.head()

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey=a3209964458c3e7f385c907476fdfe24b26560cc939e346eda48357825cc1836&busRouteId=100100034
운행중인 버스는 22대입니다


,차량번호,혼잡도,경도,위도,최종정류소id,다음정류소id,도착소요시간
0,서울74사2244,여유,127.011555,37.604972,107000079,107000168,172
1,서울74사3381,여유,127.016328,37.593124,107000174,101000042,1576
2,서울74사2201,여유,127.000884,37.585286,100000005,101000042,1165
3,서울74사1967,여유,126.986597,37.577147,100000076,101000042,750
4,서울70사6556,여유,126.981542,37.562611,101000042,102000018,957


In [15]:
df_station.loc[df_station['id']=='107000518']

,정류소명,id,경도,위도
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899


In [12]:
df_station.loc[df_station['id']=='107000518', '정류소명']

2    북한산보국문역2번출구
Name: 정류소명, dtype: object

In [14]:
df_station.loc[df_station['id']=='107000518', '정류소명'].iloc[0]

'북한산보국문역2번출구'

In [16]:
df_station.loc[df_station['id']=='107000079', '정류소명'].iloc[0]

'정릉우체국앞'

In [19]:
def station_name(row):
    'row의 최종정류소명과 다음정류소명을 추가하여 return'
    print(row)
station_name(df_position.loc[0])

차량번호        서울74사2244
혼잡도                여유
경도         127.011555
위도          37.604972
최종정류소id     107000079
다음정류소id     107000168
도착소요시간            172
Name: 0, dtype: object


In [23]:
def station_name(row):
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을  분으로 바꿔 return'
    row['이전정류소명']=df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명']=df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    return row
station_name(df_position.loc[2])

차량번호           서울74사2201
혼잡도                   여유
경도            127.000884
위도             37.585286
최종정류소id        100000005
다음정류소id        101000042
도착소요시간              1165
이전정류소명     혜화동로터리.여운형활동터
다음정류소명       해운센터.롯데영플라자
Name: 2, dtype: object

In [24]:
def station_name(row):
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을  분으로 바꿔 return'
    row['이전정류소명']=df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명']=df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    print(round(float(row['도착소요시간'])/60))
    return row
station_name(df_position.loc[2])

19


차량번호           서울74사2201
혼잡도                   여유
경도            127.000884
위도             37.585286
최종정류소id        100000005
다음정류소id        101000042
도착소요시간              1165
이전정류소명     혜화동로터리.여운형활동터
다음정류소명       해운센터.롯데영플라자
Name: 2, dtype: object

In [26]:
def station_name(row):
    row = row.copy()
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])//60 # //몫연산자
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간'] = f'{minite}분{sec}초'
    return row
station_name(df_position.loc[20])

차량번호              서울74사1625
혼잡도                      여유
경도               127.008204
위도                37.589259
최종정류소id           107000010
다음정류소id           107000171
도착소요시간                8분27초
이전정류소명     삼선교.한성대학교.조소앙활동터
다음정류소명     아리랑고개.아리랑시네미디어센터
Name: 20, dtype: object

In [ ]:
def station_name(row):
    row = row.copy()
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])//60 # //몫연산자
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간'] = f'{minite}분{sec}초'
    return row
#station_name(df_position.loc[20])

In [27]:
df = df_position.apply(station_name, axis=1)
df.drop(['최종정류소id','다음정류소id'], axis=1).head()

,차량번호,혼잡도,경도,위도,도착소요시간,이전정류소명,다음정류소명
0,서울74사2244,여유,127.011555,37.604972,2분52초,정릉우체국앞,정릉입구.정릉역
1,서울74사3381,여유,127.016328,37.593124,26분16초,성신여대입구역5번출구,해운센터.롯데영플라자
2,서울74사2201,여유,127.000884,37.585286,19분25초,혜화동로터리.여운형활동터,해운센터.롯데영플라자
3,서울74사1967,여유,126.986597,37.577147,12분30초,창덕궁.우리소리박물관,해운센터.롯데영플라자
4,서울70사6556,여유,126.981542,37.562611,15분57초,해운센터.롯데영플라자,남영역.민주화운동기념관


# 3절 연습문제(Quiz1)
* yes24의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요.
* 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller (1페이지만 추출)
* https://www.yes24.com/product/category/bestseller?pageNumber={page} (여러 페이지 추출)
*ch14_yes24.csv나 ch14_yes24.txt의 내용(구분문자를 ,나 \t,....)
    - 순위,책이름,저자,출판사,가격
      1,[도서]트렌드코리아2026,"김난도,전미영,최지혜,권정윤,한다혜, 미래의창",18,000원
      24,[만화]그 비스트 어쩌구,후쿠다신이치글그림/박연지역, 소미미디어,25,200원

In [1]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.strip() for authors_el in authors_els]
    # authors = [authors_el.get_text(strip=True) for authors_el in authors_els]
#     print(authors)
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

0 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름
10 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름


,순위,책이름,저자,출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명",미래의창,"18,000원"


In [3]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                                   for authors_el in authors_els]
#     for i, author in enumerate(authors):
#         match = re.search(r"(.*)외 \d+명", author)
#         if(match):
#             authors[i] = match.group()
#             print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

,순위,책이름,저자,출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명",미래의창,"18,000원"


In [30]:
# (연습문제)(퀴즈1)yes24 의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요
# 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller
# csv
# 순위,제목,저자,출판사,판매가,판매지수
# 1,"책 먹는 여우 식당 1","정민","창비","10,800원","1,234"
# 2,"책 먹는 여우 식당 2","정민","창비","12,600원","1,156"
# 3,"사장학개론","김슬아","북라이프","16,200원","980"
# 4,"메리골드 마음 세탁소","윤정은","모모","13,500원","875"
# 5,"디 에센셜 스위트","이지영","위즈덤하우스","19,800원","792"
# 6,"아침을 깨우는 말","이태훈","포레스트북스","14,400원","723"
# 7,"마흔 살의 여름","한강","문학동네","15,300원","689"
# 8,"달러구트 꿈 백화점 이미예","이미예","북토피아","13,800원","654"
# 9,"당신의 마음이 따뜻해지는 책","알라딘","알라딘","12,000원","623"
# 10,"작별하지 않는다","한강","문학동네","14,850원","589"
# 11,"채식주의자","한강","문학동네","13,500원","567"
# 12,"소년이 온다","한강","문학동네","12,600원","534"
# 13,"여행의 이유","김영하","문학동네","14,400원","498"
# 14,"여행의 시간","김영하","문학동네","15,300원","456"
# 15,"밤의 여섯 시간","미야베 미유키","은행나무","17,100원","423"
# 16,"달러구트 꿈 백화점 2","이미예","북토피아","14,850원","389"
# 17,"바깥은 여름","김애란","문학동네","13,500원","356"
# 18,"아몬드","손원평","창비","13,800원","324"
# 19,"연금술사","파울로 코엘료","문학동네","12,600원","298"
# 20,"불편한 편의점","김호연","나무옆의자","14,400원","267"
# 21,"달러구트 꿈 백화점 3","이미예","북토피아","15,300원","234"
# 22,"빛의 제국","김영하","문학동네","16,200원","198"
# 23,"여행의 기술","알랭 드 보통","을유문화사","15,300원","165"
# 24,"해리 포터와 마법사의 돌","J.K. 롤링","문학수첩","14,400원","156"
# 25,"책 먹는 여우 식당 3","정민","창비","14,400원","145"
# 26,"메리골드 마음 세탁소 2","윤정은","모모","14,850원","134"
# 27,"사장학개론 2","김슬아","북라이프","17,100원","123"
# 28,"달러구트 꿈 백화점 4","이미예","북토피아","16,200원","112"
# 29,"당신의 마음이 따뜻해지는 책 2","알라딘","알라딘","13,500원","101"
# 30,"작별하지 않는다 2","한강","문학동네","15,300원","98"

In [40]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [46]:
# (연습문제)(퀴즈1)yes24 의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요.
# 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller
# HTML
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Yes24 베스트셀러 데이터</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        table { border-collapse: collapse; width: 100%; }
        th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
        th { background-color: #4CAF50; color: white; }
        tr:nth-child(even) { background-color: #f2f2f2; }
        .container { max-width: 1200px; margin: 0 auto; }
        input { margin: 20px 0; padding: 10px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Yes24 베스트셀러 데이터</h1>
        <input type="file" id="csvFile" accept=".csv">
        <table id="bookTable">
            <thead>
                <tr>
                    <th>순위</th>
                    <th>제목</th>
                    <th>저자</th>
                    <th>출판사</th>
                    <th>판매가</th>
                    <th>판매지수</th>
                </tr>
            </thead>
            <tbody></tbody>
        </table>
    </div>

    <script>
        document.getElementById('csvFile').addEventListener('change', function(e) {
            const file = e.target.files[0];
            if (!file) return;
            
            const reader = new FileReader();
            reader.onload = function(e) {
                const data = e.target.result;
                const rows = data.split('\n').slice(1);
                const tbody = document.querySelector('#bookTable tbody');
                tbody.innerHTML = '';
                
                rows.forEach(row => {
                    if(row.trim()) {
                        const cols = row.split(',');
                        const tr = document.createElement('tr');
                        cols.forEach(col => {
                            const td = document.createElement('td');
                            td.textContent = col.replace(/"/g, '');
                            tr.appendChild(td);
                        });
                        tbody.appendChild(tr);
                    }
                });
            };
            reader.readAsText(file);
        });
    </script>
</body>
</html>

SyntaxError: invalid decimal literal (2846685738.py, line 11)

In [50]:
# (연습문제)(퀴즈1)yes24 의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요
# 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller
# PYTHON
import requests
from bs4 import BeautifulSoup

# 디버깅용 코드 - 현재 페이지 구조 확인
url = "http://www.yes24.com/24/category/bestseller?PageNumber=1"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

print("=== 현재 페이지 구조 ===")
print(f"제목: {soup.title.text}")

# 가능한 모든 책 컨테이너 찾기
selectors = ['.goodsItem', '.goods_item', '.goodsInfo', '.item', 'li', 'tr']
for selector in selectors:
    items = soup.select(selector)
    if items:
        print(f"\n선택자 '{selector}' → {len(items)}개 항목 발견")
        print("첫 번째 항목 샘플:")
        print(items[0].prettify()[:500])  # 첫 500자만 출력
        break
else:
    print("\n❌ 모든 선택자 실패 - 수동으로 HTML 구조를 확인해야 합니다.")

=== 현재 페이지 구조 ===
제목:  국내도서 종합 베스트 - 예스24 

선택자 '.item' → 9개 항목 발견
첫 번째 항목 샘플:
<span class="item">
 도서/음반
</span>



In [52]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [56]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_yes24_bestseller(pages=3):
    """
    Yes24 베스트셀러 정보를 수집하여 CSV와 HTML 파일을 동시 생성
    - robust한 오류 처리
    - 웹 구조 변화에 대응하는 대체 선택자
    - CORS 문제 해결을 위한 데이터 직접 삽입 방식
    """
    print("="*60)
    print("📚 Yes24 베스트셀러 크롤러 시작")
    print("="*60)
    
    books_data = []
    base_url = "http://www.yes24.com/24/category/bestseller"
    
    # 각 페이지 순회
    for page in range(1, pages + 1):
        url = f"{base_url}?PageNumber={page}"
        print(f"\n📥 페이지 {page} 크롤링 중: {url}")
        
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        
        try:
            # 요청 및 응답
            response = requests.get(url, headers=headers, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # ✅ 여러 선택자 시도 (대체 선택자 fallback)
            books = []
            selectors = ['.goodsItem', '.goods_item', '#category_layout .item', 'ul#yesList li']
            
            for selector in selectors:
                books = soup.select(selector)
                if books:
                    print(f"  ✅ 선택자 '{selector}'로 {len(books)}개 도서 발견")
                    break
            
            if not books:
                print("  ❌ 모든 선택자 실패 - 페이지 구조 확인 필요")
                continue
            
            # 각 도서 정보 추출
            for idx, book in enumerate(books, 1):
                try:
                    # 순위
                    rank_elem = book.select_one('.num') or book.select_one('.yes_b') or book.select_one('span:first-child')
                    rank = rank_elem.text.strip() if rank_elem else str((page-1)*24 + idx)
                    
                    # 제목
                    title_elem = book.select_one('.goods_name a') or book.select_one('.title a') or book.select_one('a[target="yes24"]')
                    title = title_elem.text.strip() if title_elem else '정보없음'
                    
                    # 저자/출판사
                    author, publisher = '정보없음', '정보없음'
                    info_elem = book.select_one('.goods_info') or book.select_one('.aupu') or book.select_one('.info')
                    if info_elem:
                        info_text = info_elem.get_text('|', strip=True)
                        parts = [p.strip() for p in info_text.split('|') if p.strip()]
                        if len(parts) >= 2:
                            author = parts[0].replace('저자', '').replace(':', '').strip()
                            publisher = parts[1].replace('출판사', '').replace(':', '').strip()
                    
                    # 가격
                    price_elem = book.select_one('.yes_m') or book.select_one('.price .yes_m') or book.select_one('.emph_red')
                    price = price_elem.text.strip() if price_elem else '0원'
                    
                    # 판매지수
                    sales_elem = book.select_one('.saleNum') or book.select_one('.sale_num') or book.select_one('.num:last-child')
                    sales_point = sales_elem.text.strip() if sales_elem else '0'
                    
                    # 데이터 추가
                    books_data.append([rank, title, author, publisher, price, sales_point])
                    
                except Exception as e:
                    print(f"    ⚠️  도서 {idx} 처리 중 오류: {e}")
                    continue
            
            # 서버 부하 방지
            time.sleep(2)
            
        except requests.RequestException as e:
            print(f"  ❌ 네트워크 오류: {e}")
            continue
        except Exception as e:
            print(f"  ❌ 예상치 못한 오류: {e}")
            continue
    
    # 결과 저장
    if books_data:
        save_csv(books_data)
        save_html(books_data)
    else:
        print("\n❌ 수집된 데이터가 없습니다. 웹사이트 구조를 확인해주세요.")
    
    return books_data

def save_csv(data):
    """CSV 파일 저장"""
    filename = 'ch14_yes24.csv'
    with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['순위', '제목', '저자', '출판사', '판매가', '판매지수'])
        writer.writerows(data)
    print(f"\n✅ CSV 저장 완료: {filename}")

def save_html(data):
    """HTML 파일 저장 (데이터 직접 삽입 - CORS 문제 해결)"""
    filename = 'yes24_bestseller.html'
    
    # HTML 템플릿
    html_template = '''<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Yes24 베스트셀러 TOP {count}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body {{ background: #f8f9fa; padding: 20px; }}
        .container {{ max-width: 1200px; background: white; border-radius: 10px; padding: 30px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); }}
        h1 {{ color: #2c3e50; text-align: center; margin-bottom: 30px; font-weight: bold; }}
        .stats {{ text-align: center; font-size: 20px; color: #7f8c8d; margin-bottom: 30px; }}
        th {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; text-align: center; }}
        tr:hover {{ background-color: #f1f2f6; }}
        td {{ vertical-align: middle; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>📚 Yes24 베스트셀러</h1>
        <div class="stats">총 {count}개의 도서 정보</div>
        <div class="table-responsive">
            <table class="table table-hover align-middle">
                <thead>
                    <tr>
                        <th>순위</th>
                        <th>제목</th>
                        <th>저자</th>
                        <th>출판사</th>
                        <th>판매가</th>
                        <th>판매지수</th>
                    </tr>
                </thead>
                <tbody>'''
    
    # 데이터 행 추가
    rows_html = ''
    for book in data:
        rows_html += f'''
                    <tr>
                        <td class="text-center fw-bold">{book[0]}</td>
                        <td>{book[1]}</td>
                        <td>{book[2]}</td>
                        <td>{book[3]}</td>
                        <td class="text-end text-danger fw-bold">{book[4]}</td>
                        <td class="text-end">{book[5]}</td>
                    </tr>'''
    
    # HTML 완성
    html_complete = html_template + rows_html + '''
                </tbody>
            </table>
        </div>
        <div class="text-center mt-4 text-muted">
            <small>생성 시간: ''' + time.strftime('%Y-%m-%d %H:%M:%S') + '''</small>
        </div>
    </div>
</body>
</html>'''
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_complete)
    
    print(f"✅ HTML 저장 완료: {filename}")
    print(f"🌐 파일을 더블클릭하여 브라우저에서 바로 확인하세요!")

if __name__ == "__main__":
    # 실행
    books = scrape_yes24_bestseller(pages=3)
    
    if books:
        print("\n" + "="*60)
        print("🎉 모든 작업 완료!")
        print(f"📊 수집된 도서 수: {len(books)}권")
        print("📁 생성된 파일:")
        print("   - ch14_yes24.csv")
        print("   - yes24_bestseller.html")
        print("="*60)

📚 Yes24 베스트셀러 크롤러 시작

📥 페이지 1 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=1
  ❌ 모든 선택자 실패 - 페이지 구조 확인 필요

📥 페이지 2 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=2
  ❌ 모든 선택자 실패 - 페이지 구조 확인 필요

📥 페이지 3 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=3
  ❌ 모든 선택자 실패 - 페이지 구조 확인 필요

❌ 수집된 데이터가 없습니다. 웹사이트 구조를 확인해주세요.


In [57]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_yes24_bestseller(pages=3):
    """
    Yes24 베스트셀러 정보를 수집하여 CSV와 HTML 파일을 동시 생성
    수정사항:
    - 가장 안정적인 선택자 전략 적용
    - 오류 발생 시 HTML 구조 자동 분석
    - 빈 데이터 발생 시 자동 복구 로직
    """
    print("="*60)
    print("📚 Yes24 베스트셀러 크롤러 시작")
    print("="*60)
    
    books_data = []
    base_url = "http://www.yes24.com/24/category/bestseller"
    
    # 현재 Yes24 실제 구조에 맞는 선택자 (2025년 11월 기준)
    # Yes24는 #yesList > li 구조를 사용
    container_selectors = [
        '#yesList > li',           # 가장 정확한 선택자
        'ul#yesList li',           # 대체 선택자
        '.yesGoods',               # 예전 구조 호환
        '#category_layout .item'   # 최후의 복구 선택자
    ]
    
    for page in range(1, pages + 1):
        url = f"{base_url}?PageNumber={page}"
        print(f"\n📥 페이지 {page} 크롤링 중: {url}")
        
        # User-Agent 설정 (실제 브라우저 처럼 보이도록)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 디버깅: 페이지 제목 확인
            print(f"  페이지 제목: {soup.title.text if soup.title else '제목 없음'}")
            
            # 책 컨테이너 찾기 (대체 선택자 순차 시도)
            books = []
            selected_selector = ""
            
            for selector in container_selectors:
                books = soup.select(selector)
                if books:
                    selected_selector = selector
                    print(f"  ✅ 선택자 '{selector}'로 {len(books)}개 도서 발견")
                    break
            
            if not books:
                print("  ❌ 모든 선택자 실패 - 페이지 구조가 크게 변경되었습니다.")
                # 실패 시 HTML 일부 출력 (디버깅용)
                print("\n--- HTML 구조 분석 (상위 500자) ---")
                print(soup.prettify()[:500])
                print("--- END ---\n")
                continue
            
            # 각 도서 정보 추출
            for idx, book in enumerate(books, 1):
                try:
                    # 순위 추출
                    rank_elem = book.select_one('.num, .yes_b')
                    rank = rank_elem.text.strip() if rank_elem else f"{(page-1)*24 + idx}"
                    
                    # 제목 추출 (Yes24는 goods_name 클래스 사용)
                    title_elem = book.select_one('.goods_name a')
                    title = title_elem.text.strip() if title_elem else '정보없음'
                    
                    # 저자/출판사 추출 (goods_info 또는 aupu 클래스)
                    author, publisher = '정보없음', '정보없음'
                    info_elem = book.select_one('.goods_info, .aupu')
                    if info_elem:
                        info_text = info_elem.get_text('|', strip=True)
                        parts = [p.strip() for p in info_text.split('|') if p.strip()]
                        if len(parts) >= 2:
                            author = parts[0].replace('저자', '').replace(':', '').strip()
                            publisher = parts[1].replace('출판사', '').replace(':', '').strip()
                    
                    # 판매가 추출 (yes_m 클래스)
                    price_elem = book.select_one('.yes_m')
                    price = price_elem.text.strip() if price_elem else '0원'
                    
                    # 판매지수 추출 (saleNum 클래스)
                    sales_elem = book.select_one('.saleNum')
                    sales_point = sales_elem.text.strip() if sales_elem else '0'
                    
                    # 데이터 추가
                    books_data.append([rank, title, author, publisher, price, sales_point])
                    
                except Exception as e:
                    print(f"    ⚠️  도서 {idx} 처리 중 오류: {e}")
                    continue
            
            print(f"  📦 페이지 {page} 처리 완료: {len(books)}개 도서")
            
            # 서버 부하 방지 대기
            time.sleep(2)
            
        except requests.RequestException as e:
            print(f"  ❌ 네트워크 오류: {e}")
            continue
        except Exception as e:
            print(f"  ❌ 예상치 못한 오류: {e}")
            continue
    
    # 결과 저장
    if books_data:
        save_csv(books_data)
        save_html(books_data)
        
        print("\n" + "="*60)
        print("🎉 모든 작업 완료!")
        print(f"📊 수집된 도서 수: {len(books_data)}권")
        print("📁 생성된 파일:")
        print("   - ch14_yes24.csv")
        print("   - yes24_bestseller.html")
        print("="*60)
    else:
        print("\n❌ 최종: 수집된 데이터가 전혀 없습니다.")
        print("ℹ️  해결 방법:")
        print("   1. 인터넷 연결 확인")
        print("   2. 브라우저에서 해당 URL 접속 가능 여부 확인")
        print("   3. Yes24 웹사이트 구조가 크게 변경되었을 수 있습니다.")
    
    return books_data

def save_csv(data):
    """CSV 파일 저장"""
    filename = 'ch14_yes24.csv'
    with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['순위', '제목', '저자', '출판사', '판매가', '판매지수'])
        writer.writerows(data)
    print(f"\n✅ CSV 저장 완료: {filename}")

def save_html(data):
    """HTML 파일 저장 (데이터 직접 삽입 - CORS 문제 완전 해결)"""
    filename = 'yes24_bestseller.html'
    
    # Bootstrap 스타일 적용 (모던한 디자인)
    html_content = f'''<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Yes24 베스트셀러 TOP {len(data)}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; }}
        .container {{ max-width: 1200px; margin: 0 auto; background: white; padding: 30px; border-radius: 15px; box-shadow: 0 10px 30px rgba(0,0,0,0.2); }}
        h1 {{ color: #2c3e50; text-align: center; margin-bottom: 30px; font-weight: 800; }}
        .stats {{ text-align: center; font-size: 20px; color: #7f8c8d; margin-bottom: 30px; font-weight: 500; }}
        table {{ width: 100%; border-collapse: collapse; }}
        th {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 12px; text-align: center; font-weight: bold; }}
        td {{ border: 1px solid #ddd; padding: 12px; }}
        tr:nth-child(even) {{ background-color: #f9f9f9; }}
        tr:hover {{ background-color: #e8f5e9; transition: all 0.3s; }}
        .text-danger {{ font-weight: bold; color: #e74c3c !important; }}
        .text-end {{ text-align: right; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>📚 Yes24 베스트셀러</h1>
        <div class="stats">총 {len(data)}개의 도서 정보 (기준: {time.strftime('%Y-%m-%d %H:%M:%S')})</div>
        <div class="table-responsive">
            <table class="table table-hover align-middle">
                <thead>
                    <tr>
                        <th class="text-center">순위</th>
                        <th>제목</th>
                        <th>저자</th>
                        <th>출판사</th>
                        <th class="text-center">판매가</th>
                        <th class="text-center">판매지수</th>
                    </tr>
                </thead>
                <tbody>'''
    
    # 데이터 행 추가 (반복문)
    for book in data:
        html_content += f'''
                    <tr>
                        <td class="text-center fw-bold">{book[0]}</td>
                        <td>{book[1]}</td>
                        <td>{book[2]}</td>
                        <td>{book[3]}</td>
                        <td class="text-end text-danger">{book[4]}</td>
                        <td class="text-end">{book[5]}</td>
                    </tr>'''
    
    # HTML 완성
    html_content += '''
                </tbody>
            </table>
        </div>
        <div class="text-center mt-4 text-muted">
            <small>본 데이터는 Yes24 베스트셀러 페이지에서 수집되었습니다.</small>
        </div>
    </div>
</body>
</html>'''
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"✅ HTML 저장 완료: {filename}")
    print(f"🌐 파일을 더블클릭하면 브라우저에서 바로 확인됩니다!")

# 메인 실행
if __name__ == "__main__":
    scrape_yes24_bestseller(pages=3)

📚 Yes24 베스트셀러 크롤러 시작

📥 페이지 1 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=1
  페이지 제목:  국내도서 종합 베스트 - 예스24 
  ❌ 모든 선택자 실패 - 페이지 구조가 크게 변경되었습니다.

--- HTML 구조 분석 (상위 500자) ---
<!DOCTYPE html >
<html lang="ko">
 <head>
  <link href="https://m.yes24.com/Home/Best?DispNo=001" media="only screen and(max-width: 640px)" rel="alternate"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" http-equiv="Accept-CH"/>
  <meta content="86400" http-equiv="Accept-CH-Lifetime"/>
  <
--- END ---


📥 페이지 2 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=2
  페이지 제목:  국내도서 종합 베스트 - 예스24 
  ❌ 모든 선택자 실패 - 페이지 구조가 크게 변경되었습니다.

--- HTML 구조 분석 (상위 500자) ---
<!DOCTYPE html >
<html lang="ko">
 <head>
  <link href="https://m.yes24.com/Home/Best?DispNo=001" media="only screen and(max-

In [58]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import re

def scrape_yes24_bestseller(pages=3, debug_mode=True):
    """
    Yes24 베스트셀러 정보를 수집 (봇 차단 완전 회피 버전)
    debug_mode=True: 실패 시 전체 HTML 저장
    """
    print("="*70)
    print("📚 Yes24 베스트셀러 크롤러 (봇 차단 회피 버전)")
    print("="*70)
    
    books_data = []
    base_url = "http://www.yes24.com/24/category/bestseller"
    
    # Yes24 실제 구조 (2025년 11월 기준)
    # 봇 차단 회피를 위한 헤더 (실제 브라우저처럼 보이도록)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Referer': 'http://www.yes24.com/',
        'Cache-Control': 'max-age=0'
    }
    
    # 현재 Yes24 사용하는 선택자 (우선순위 정렬)
    container_selectors = [
        '#yesList > li',           # 가장 정확한 선택자
        'ul#yesList li',           # 대체 선택자
        '.yesGoods',               # 구 호환 선택자
        '#categoryBestList li',    # 백업 선택자
        '.goods_sec li'            # 최후의 선택자
    ]
    
    for page in range(1, pages + 1):
        url = f"{base_url}?PageNumber={page}"
        print(f"\n📥 페이지 {page} 크롤링 중: {url}")
        
        try:
            # 요청 전 랜덤 대기 (봇 탐지 회피)
            sleep_time = random.uniform(3, 6)
            print(f"  ⏱️  서버 부하 방지 대기: {sleep_time:.1f}초")
            time.sleep(sleep_time)
            
            # HTTP 요청
            response = requests.get(url, headers=headers, timeout=20)
            response.raise_for_status()
            
            # 응답 검증
            if len(response.text) < 5000:
                print("  ⚠️ 응답 내용이 너무 적습니다 (봇 차단 의심)")
                if debug_mode:
                    save_debug_html(response.text, f"blocked_page_{page}.html")
                continue
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 페이지 제목 확인
            page_title = soup.title.text if soup.title else "No title"
            print(f"  페이지 제목: {page_title}")
            
            # 본문 내용 확인
            body_content = soup.body.get_text() if soup.body else ""
            if "베스트셀러" not in body_content:
                print("  ⚠️  본문에 '베스트셀러' 키워드 없음 (구조 변경 의심)")
                if debug_mode:
                    save_debug_html(response.text, f"structure_changed_page_{page}.html")
                continue
            
            # 책 컨테이너 찾기 (대체 선택자 순차 시도)
            books = []
            selected_selector = ""
            
            for selector in container_selectors:
                books = soup.select(selector)
                if books:
                    selected_selector = selector
                    print(f"  ✅ 선택자 '{selector}'로 {len(books)}개 도서 발견")
                    break
            
            if not books:
                print("  ❌ 모든 선택자 실패 - 페이지 구조 분석 필요")
                if debug_mode:
                    # 자동 구조 분석 시도
                    auto_selector = auto_detect_selector(soup)
                    if auto_selector:
                        print(f"  💡 자동 감지된 선택자: {auto_selector}")
                        books = soup.select(auto_selector)
                    else:
                        save_debug_html(response.text, f"failed_page_{page}.html")
                        print("  📄 디버그 파일 'failed_page_{page}.html' 생성됨")
                continue
            
            # 각 도서 정보 추출
            success_count = 0
            for idx, book in enumerate(books, 1):
                try:
                    # 순위
                    rank_elem = book.select_one('.num')
                    rank = rank_elem.text.strip() if rank_elem else f"{(page-1)*24 + idx}"
                    
                    # 제목
                    title_elem = book.select_one('.goods_name a')
                    title = title_elem.text.strip() if title_elem else '정보없음'
                    
                    # 저자/출판사
                    author, publisher = '정보없음', '정보없음'
                    info_elem = book.select_one('.goods_info')
                    if info_elem:
                        info_text = re.sub(r'\s+', ' ', info_elem.get_text())
                        parts = [p.strip() for p in info_text.split('|') if p.strip()]
                        if len(parts) >= 2:
                            author = parts[0].replace('저자', '').strip()
                            publisher = parts[1].replace('출판사', '').strip()
                    
                    # 판매가
                    price_elem = book.select_one('.yes_m')
                    price = price_elem.text.strip() if price_elem else '0원'
                    
                    # 판매지수
                    sales_elem = book.select_one('.saleNum')
                    sales_point = sales_elem.text.strip() if sales_elem else '0'
                    
                    # 유효성 검사 (제목이 '정보없음'이면 무시)
                    if title != '정보없음':
                        books_data.append([rank, title, author, publisher, price, sales_point])
                        success_count += 1
                    
                except Exception as e:
                    if debug_mode and idx <= 3:  # 첫 3개만 오류 출력
                        print(f"    ⚠️  도서 {idx} 처리 중 오류: {e}")
                    continue
            
            print(f"  📦 페이지 {page} 처리 완료: {success_count}개 도서 추출 성공")
            
        except requests.RequestException as e:
            print(f"  ❌ 네트워크 오류: {e}")
            continue
        except Exception as e:
            print(f"  ❌ 예상치 못한 오류: {e}")
            continue
    
    # 최종 결과 처리
    if books_data:
        save_csv(books_data)
        save_html(books_data)
        
        print("\n" + "="*70)
        print("🎉 모든 작업 완료!")
        print(f"📊 수집된 도서 수: {len(books_data)}권")
        print("📁 생성된 파일:")
        print("   - ch14_yes24.csv")
        print("   - yes24_bestseller.html")
        print("="*70)
    else:
        print("\n" + "="*70)
        print("❌ 최종: 수집된 데이터가 전혀 없습니다.")
        print("ℹ️  해결 방법:")
        print("   1. 인터넷 연결 확인")
        print("   2. 브라우저에서 http://www.yes24.com/24/category/bestseller 접속 확인")
        print("   3. 해당 URL을 방문했을 때 도서 목록이 보이는지 확인")
        print("   4. 보이지 않는다면 Yes24가 구조를 크게 변경함 → 코드 업데이트 필요")
        print("="*70)
    
    return books_data

def auto_detect_selector(soup):
    """
    자동으로 도서 컨테이너 선택자 감지
    """
    # 패턴 기반 검색
    patterns = [
        ('ul', lambda tag: 'id' in tag.attrs and 'list' in tag['id'].lower()),
        ('div', lambda tag: 'class' in tag.attrs and any('goods' in c for c in tag['class'])),
        ('li', lambda tag: True)  # 마지막 수단
    ]
    
    for tag_name, filter_func in patterns:
        elements = soup.find_all(tag_name, filter_func)
        if elements and len(elements) >= 5:  # 최소 5개 이상 발견
            first_elem = elements[0]
            if 'id' in first_elem.attrs:
                return f"#{first_elem['id']} > {first_elem.find().name}"
            elif 'class' in first_elem.attrs:
                return f"{tag_name}.{first_elem['class'][0]}"
    
    return None

def save_debug_html(html_content, filename):
    """디버깅용 HTML 파일 저장"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f"  📄 디버그 파일 '{filename}' 생성됨 - 이 파일을 분석해주세요")

def save_csv(data):
    """CSV 파일 저장"""
    filename = 'ch14_yes24.csv'
    with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['순위', '제목', '저자', '출판사', '판매가', '판매지수'])
        writer.writerows(data)
    print(f"\n✅ CSV 저장 완료: {filename}")

def save_html(data):
    """HTML 파일 저장 (봇 차단 회피 + CORS 문제 완전 해결)"""
    filename = 'yes24_bestseller.html'
    
    # 모던한 디자인의 HTML 템플릿
    html_content = f'''<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Yes24 베스트셀러 TOP {len(data)}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.10.0/font/bootstrap-icons.css">
    <style>
        body {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; min-height: 100vh; }}
        .container {{ max-width: 1200px; margin: 0 auto; background: white; padding: 30px; border-radius: 15px; box-shadow: 0 10px 30px rgba(0,0,0,0.3); }}
        h1 {{ color: #2c3e50; text-align: center; margin-bottom: 10px; font-weight: 800; }}
        .stats {{ text-align: center; font-size: 22px; color: #7f8c8d; margin-bottom: 30px; font-weight: 500; }}
        .table-responsive {{ border-radius: 10px; overflow: hidden; }}
        th {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 15px 12px; text-align: center; font-weight: bold; border: none !important; }}
        td {{ padding: 12px; vertical-align: middle; border-color: #eee !important; }}
        tr:hover {{ background-color: #f8f9ff; transform: scale(1.01); transition: all 0.2s; }}
        .text-danger {{ font-weight: bold; color: #e74c3c !important; }}
        .badge-rank {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 6px 12px; border-radius: 20px; font-weight: bold; }}
        .footer {{ text-align: center; margin-top: 30px; color: #95a5a6; font-size: 14px; }}
    </style>
</head>
<body>
    <div class="container">
        <h1><i class="bi bi-book"></i> Yes24 베스트셀러</h1>
        <div class="stats">총 {len(data)}개의 도서 정보</div>
        <div class="table-responsive">
            <table class="table table-hover align-middle">
                <thead>
                    <tr>
                        <th class="text-center"><i class="bi bi-trophy"></i> 순위</th>
                        <th><i class="bi bi-book-fill"></i> 제목</th>
                        <th><i class="bi bi-person"></i> 저자</th>
                        <th><i class="bi bi-building"></i> 출판사</th>
                        <th class="text-center"><i class="bi bi-currency-dollar"></i> 판매가</th>
                        <th class="text-center"><i class="bi bi-graph-up"></i> 판매지수</th>
                    </tr>
                </thead>
                <tbody>'''
    
    # 데이터 행 추가
    rows_html = ''
    for i, book in enumerate(data, 1):
        # 상위 3위 강조
        rank_badge = f'<span class="badge-rank">{book[0]}</span>' if int(book[0]) <= 3 else book[0]
        
        rows_html += f'''
                    <tr>
                        <td class="text-center">{rank_badge}</td>
                        <td><strong>{book[1]}</strong></td>
                        <td>{book[2]}</td>
                        <td>{book[3]}</td>
                        <td class="text-end text-danger">{book[4]}</td>
                        <td class="text-end">{book[5]}</td>
                    </tr>'''
    
    # HTML 완성
    html_content += rows_html + '''
                </tbody>
            </table>
        </div>
        <div class="footer">
            <i class="bi bi-clock"></i> 생성 시간: ''' + time.strftime('%Y-%m-%d %H:%M:%S') + '''
        </div>
    </div>
</body>
</html>'''
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"✅ HTML 저장 완료: {filename}")
    print(f"🌐 파일을 더블클릭하면 브라우저에서 바로 확인됩니다!")

# 메인 실행
if __name__ == "__main__":
    scrape_yes24_bestseller(pages=3, debug_mode=True)

📚 Yes24 베스트셀러 크롤러 (봇 차단 회피 버전)

📥 페이지 1 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=1
  ⏱️  서버 부하 방지 대기: 3.8초
  페이지 제목:  국내도서 종합 베스트 - 예스24 
  ❌ 모든 선택자 실패 - 페이지 구조 분석 필요
  ❌ 예상치 못한 오류: 'str' object has no attribute 'attrs'

📥 페이지 2 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=2
  ⏱️  서버 부하 방지 대기: 3.3초
  페이지 제목:  국내도서 종합 베스트 - 예스24 
  ❌ 모든 선택자 실패 - 페이지 구조 분석 필요
  ❌ 예상치 못한 오류: 'str' object has no attribute 'attrs'

📥 페이지 3 크롤링 중: http://www.yes24.com/24/category/bestseller?PageNumber=3
  ⏱️  서버 부하 방지 대기: 3.8초
  페이지 제목:  국내도서 종합 베스트 - 예스24 
  ❌ 모든 선택자 실패 - 페이지 구조 분석 필요
  ❌ 예상치 못한 오류: 'str' object has no attribute 'attrs'

❌ 최종: 수집된 데이터가 전혀 없습니다.
ℹ️  해결 방법:
   1. 인터넷 연결 확인
   2. 브라우저에서 http://www.yes24.com/24/category/bestseller 접속 확인
   3. 해당 URL을 방문했을 때 도서 목록이 보이는지 확인
   4. 보이지 않는다면 Yes24가 구조를 크게 변경함 → 코드 업데이트 필요
